In [7]:
import json

# 🔥 PASTE YOUR TOKEN INSIDE THE QUOTES BELOW
token = "KGAT_1046dbac362f5e84561e77d5e5a066f7"

kaggle_dict = {
    "username": "haroon7878",
    "key": token
}

with open("/content/kaggle.json", "w") as f:
    json.dump(kaggle_dict, f)

print("✔ kaggle.json file created!")


✔ kaggle.json file created!


In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✔ kaggle.json installed successfully!")


In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia


In [ ]:
!unzip chest-xray-pneumonia.zip


In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!mv /content/chest_xray /content/drive/MyDrive/chest_xray_new


In [ ]:
import os
os.listdir('/content/drive/MyDrive')


In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 224
batch_size = 32

base_path = "/content/drive/MyDrive/chest_xray_new"

train_path = base_path + "/train"
val_path   = base_path + "/val"
test_path  = base_path + "/test"

train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen   = ImageDataGenerator(rescale=1.0/255)
test_datagen  = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    val_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)

test_gen = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


BUILD THE MODEL


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Load MobileNetV2 base model (pretrained on ImageNet)
base_model = MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
base_model.trainable = False   # Freeze layers to speed up training

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


MODEL TRAINING

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)


EVALUATE ON TEST DATA


In [ ]:
test_loss, test_acc = model.evaluate(test_gen)
print("Test Accuracy:", test_acc)
print("Test Loss:", test_loss)


UPLOAD AN XRAY IMAGE AND TEST


fine tune the base model

In [23]:
# Unfreeze last 30 layers of MobileNetV2
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=3
)


Epoch 1/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 409s 2s/step - accuracy: 0.9106 - loss: 0.2972 - val_accuracy: 0.8750 - val_loss: 0.2035
Epoch 2/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 400s 2s/step - accuracy: 0.9665 - loss: 0.0844 - val_accuracy: 0.8750 - val_loss: 0.1818
Epoch 3/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 388s 2s/step - accuracy: 0.9799 - loss: 0.0562 - val_accuracy: 0.8750 - val_loss: 0.1703


improve validation set

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.15   # 15% validation
)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


In [25]:
class_weight = {0: 1.0, 1: 0.7}


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    class_weight=class_weight
)


graphs

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# -----------------------------
# 📌 1. Accuracy Graph
# -----------------------------
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Model Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# -----------------------------
# 📌 2. Loss Graph
# -----------------------------
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Model Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()


# -----------------------------
# 📌 3. Classification Report
# -----------------------------
test_pred = model.predict(test_gen)
test_pred_classes = (test_pred > 0.5).astype("int32")

print("\n📌 Classification Report:")
print(classification_report(test_gen.classes, test_pred_classes, target_names=["NORMAL", "PNEUMONIA"]))


# -----------------------------
# 📌 4. Confusion Matrix
# -----------------------------
cm = confusion_matrix(test_gen.classes, test_pred_classes)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["NORMAL", "PNEUMONIA"],
            yticklabels=["NORMAL", "PNEUMONIA"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


# -----------------------------
# 📌 5. Final Summary
# -----------------------------
print("\n==============================")
print("📌 FINAL MODEL SUMMARY")
print("==============================")

print(f"Train Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

print("\nModel fully trained, evaluated, and ready for project submission.")


In [ ]:
model.save('/content/drive/MyDrive/pneumonia_model.h5')


In [34]:
model.save('/content/drive/MyDrive/pneumonia_model.keras')
